In [12]:
import credentials
import settings

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

class MyStreamerListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return False
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)
        sentiment = TextBlob(status.text)
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count = status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude= status.coordinates['coordinates']
            latitude = status.coordinates['coordinates']
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        print(status.text)
        print("Long: {}, Lat: {}". format(longitude, latitude))
        
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location,user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql,val)
            mydb.commit()
            mycursor.close()


In [13]:
def clean_tweet(self, tweet): 

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
                                
def deEmojify(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [14]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="gaurav",
    database="TwitterDB",
    charset = 'utf8'
)

if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()


ProgrammingError: 1050 (42S01): Table 'starbucks' already exists

In [15]:
auth  = tweepy.OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [16]:
import time
x=6
while(x == 6):
    myStreamerListener = MyStreamerListener()
    myStream = tweepy.Stream(auth = api.auth, listener = myStreamerListener)
    myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
    time.sleep(10)
mydb.close()

@thesvenprint why cant u just go to starbucks... i dont get it
Long: None, Lat: None
RT @Adorable_Cats_: Met a cute barista at Starbucks today 😻 https://t.co/WyvqTkHeCw
Long: None, Lat: None
RT @BloombergTV: Starbucks CEO breaks down how they are re-opening stores in China ☕ https://t.co/sVJntuCbrA https://t.co/4mfWPPrUnu
Long: None, Lat: None
Starbucks CEO Says 'Stir-Crazy' Consumers Will Return But It'll Take Time
https://t.co/0iN8ktofYp
Long: None, Lat: None
RT @zvabi: missing starbucks more than anything
Long: None, Lat: None
Why Starbucks never took off in South Africa. 

#thread
Long: None, Lat: None
@Starbucks I hate tea
Long: None, Lat: None
RT @22_Minutes: Starbucks will reopen locations across Canada this week, but their dining areas will remain closed. Sorry everyone, your sc…
Long: None, Lat: None
RT @zvabi: missing starbucks more than anything
Long: None, Lat: None
RT @cinnderellaaa_: Bitches posting "stay home" and have Starbucks on their story every day 😂
Long: None, Lat

MySQLInterfaceError: Python type list cannot be converted